In [1]:
from __future__ import print_function
import keras
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
import os
import keras_tuner

In [2]:
batch_size = 32
num_classes = 1000
epochs = 10

In [3]:
(x_train, y_train), (x_test, y_test) =  keras.datasets.cifar10.load_data()

In [4]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

In [5]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [6]:
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
test_generator = datagen.flow(x_test, y_test, batch_size=batch_size)

In [7]:
model = Sequential()
model.add(tensorflow.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(32, 32, 3), padding="same"))
model.add(tensorflow.keras.layers.Dropout( rate=0.2))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(tensorflow.keras.layers.Dropout( rate=0.5))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(tensorflow.keras.layers.Flatten())
model.add(tensorflow.keras.layers.Dense(512, activation="relu"))
model.add(tensorflow.keras.layers.Dense(64, activation="tanh"))
model.add(tensorflow.keras.layers.Dense(10, activation="softmax"))   

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

In [9]:
model.fit(train_generator,

                    epochs=epochs,
                    validation_data=test_generator,
                    )

Epoch 1/10
 482/1563 [========>.....................] - ETA: 1:18 - loss: 103.7672 - accuracy: 0.0886

KeyboardInterrupt: 

In [ ]:
score = model.evaluate_generator(test_generator, steps=len(x_test) // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def build_model(hp):
    n_hiddenc1 = hp.Int("n_hiddenc1", min_value=0, max_value=8 )  
    n_hiddenc2 = hp.Int("n_hiddenc2", min_value=0, max_value=8 )   
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8 )
    n_neurons = hp.Int("n_neurons", min_value=32, max_value=256 )
    size=hp.Int("size",min_value=3, max_value=6)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log" )
    optimizer = hp.Choice("optimizer", values=["adam", "sgd"])
    if optimizer=="adam":
       optimizer = tensorflow.keras.optimizers.Adam(learning_rate=learning_rate)
    else:  
        optimizer = tensorflow.keras.optimizers.SGD(learning_rate=learning_rate)
    model = tensorflow.keras.Sequential()
    model.add(tensorflow.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(32, 32, 3), padding="same"))
    for i in range (n_hiddenc1):
        model.add(tensorflow.keras.layers.Conv2D(n_neurons, kernel_size=(size, size), strides=(1, 1), activation='tanh', input_shape=( 32, 32, 3), padding="same"))
        model.add(tensorflow.keras.layers.AveragePooling2D(pool_size=(size-1, size-1), strides=(1, 1), padding='valid'))

    for i in range (n_hiddenc2):
        model.add(tensorflow.keras.layers.Conv2D(n_neurons, kernel_size=(size, size), strides=(1, 1), activation='tanh', input_shape=( 32, 32, 3), padding="same"))
        model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=(size-1, size-1), strides=(1, 1), padding='valid'))
    model.add(tensorflow.keras.layers.Flatten())

    for i in range (n_hidden):
        model.add(tensorflow.keras.layers.Dense(n_neurons, activation="relu"))
    for i in range (n_hidden):
        model.add(tensorflow.keras.layers.Dense(n_neurons, activation="tanh"))
    model.add(tensorflow.keras.layers.Dense(10, activation="softmax"))    
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [ ]:
random_search_tuner=keras_tuner.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials = 4,
    overwrite=True,
    seed=37
)

In [ ]:
random_search_tuner.search(x_train, y_train, epochs=10, validation_split=0.2)

In [ ]:
best_model=random_search_tuner.get_best_models()[0]

In [ ]:
results = random_search_tuner.oracle.get_best_trials()[0]
results.summary()

In [ ]:
best_model.fit(train_generator,

                    epochs=epochs,
                    validation_data=test_generator,
                    )

In [ ]:
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)

In [ ]:
score = best_model.evaluate_generator(test_generator, steps=len(x_test) // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])